In [1]:
import tensorflow as tf
import numpy as np

states = [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
terminal = [False, False, False, False, True, False, False, False, False, False]
NUM_STATES = len(states)
NUM_ACTIONS = 2
DISCOUNT_FACTOR = 0.5


def one_hot_state(index):
    array = np.zeros(NUM_STATES)
    array[index] = 1.
    return array


session = tf.Session()
state = tf.placeholder("float", [None, NUM_STATES])
targets = tf.placeholder("float", [None, NUM_ACTIONS])

weights = tf.Variable(tf.constant(0., shape=[NUM_STATES, NUM_ACTIONS]))

output = tf.matmul(state, weights)

loss = tf.reduce_mean(tf.square(output - targets))
train_operation = tf.train.GradientDescentOptimizer(1.).minimize(loss)

session.run(tf.initialize_all_variables())

for _ in range(50):
    state_batch = []
    rewards_batch = []

    for state_index in range(NUM_STATES):
        state_batch.append(one_hot_state(state_index))

        minus_action_index = (state_index - 1) % NUM_STATES
        plus_action_index = (state_index + 1) % NUM_STATES

        if terminal[minus_action_index]:
            minus_action_q_value = DISCOUNT_FACTOR * states[minus_action_index]
        else:
            minus_action_state_reward = session.run(output, feed_dict={state: [one_hot_state(minus_action_index)]})
            minus_action_q_value = DISCOUNT_FACTOR * (states[minus_action_index] + np.max(minus_action_state_reward))

        if terminal[plus_action_index]:
            plus_action_q_value = DISCOUNT_FACTOR * states[plus_action_index]
        else:
            plus_action_state_reward = session.run(output, feed_dict={state: [one_hot_state(plus_action_index)]})
            plus_action_q_value = DISCOUNT_FACTOR * (states[plus_action_index] + np.max(plus_action_state_reward))

        action_rewards = [minus_action_q_value, plus_action_q_value]
        rewards_batch.append(action_rewards)

    session.run(train_operation, feed_dict={
        state: state_batch,
        targets: rewards_batch})

    print([states[x] + (1-float(terminal[x]))*np.max(session.run(output, feed_dict={state: [one_hot_state(x)]}))
           for x in range(NUM_STATES)])

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[0.0, 0.0, 0.0, 0.05000000074505806, 1.0, 0.05000000074505806, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.002500000176951289, 0.0949999988079071, 1.0, 0.0949999988079071, 0.002500000176951289, 0.0, 0.0, 0.0]
[0.0, 0.0001250000059371814, 0.007000000216066837, 0.1354999989271164, 1.0, 0.1354999989271164, 0.007000000216066837, 0.0001250000059371814, 0.0, 0.0]
[6.250000296859071e-06, 0.00046250002924352884, 0.013074999675154686, 0.17194999754428864, 1.0, 0.17194999754428864, 0.013074999675154686, 0.00046250002924352884, 6.250000296859071e-06, 0.0]
[2.8750002456945367e-05, 0.001069999998435378, 0.020364999771118164, 0.2047550082206726, 1.0, 0.2047550082206726, 0.020364999771118164, 0.001069999998435378, 2.8750002456945367e-05, 3.125000205272954e-07]
[7.937500777188689e-05, 0.001981250010430813, 0.028566250577569008, 0.23427951335906982, 1.0, 0.23427951335906982, 0.028566250577569008, 0.001981250010430813, 7.937500777188689e-05,

[0.04084056243300438, 0.10370390117168427, 0.23572279512882233, 0.4951511323451996, 1.0, 0.4951511323451996, 0.23572279512882233, 0.10370390117168427, 0.04084056243300438, 0.014149904251098633]
[0.041941702365875244, 0.10511965304613113, 0.23690807819366455, 0.4956360161304474, 1.0, 0.4956360161304474, 0.23690807819366455, 0.10511965304613113, 0.041941702365875244, 0.014776942320168018]
[0.04300351440906525, 0.10645309090614319, 0.2379990667104721, 0.4960724115371704, 1.0, 0.4960724115371704, 0.2379990667104721, 0.10645309090614319, 0.04300351440906525, 0.015396333299577236]
[0.04402581602334976, 0.10770773887634277, 0.2390027791261673, 0.49646517634391785, 1.0, 0.49646517634391785, 0.2390027791261673, 0.10770773887634277, 0.04402581602334976, 0.016006875783205032]
[0.045008622109889984, 0.10888710618019104, 0.23992575705051422, 0.4968186616897583, 1.0, 0.4968186616897583, 0.23992575705051422, 0.10888710618019104, 0.045008622109889984, 0.016607478260993958]
[0.04595211520791054, 0.1099